In [17]:
import os
import yaml
from jinja2 import Environment, PackageLoader, FileSystemLoader, StrictUndefined
from jinja2.exceptions import TemplateNotFound

In [18]:
from hokusai.lib.config_loader import ConfigLoader
from hokusai.lib.exceptions import HokusaiError

In [19]:
configloader = ConfigLoader("s3://artsy-citadel/k8s/hokusai_vars.yml")


In [20]:
config = configloader.load()
print config

{'vars': {'artsyNetWildcardSSLCert': 'arn:aws:iam::585031190124:server-certificate/2020-03-18_artsy-net-wildcard'}}


In [21]:
print(config["vars"])

{'artsyNetWildcardSSLCert': 'arn:aws:iam::585031190124:server-certificate/2020-03-18_artsy-net-wildcard'}


In [37]:
template = 'production.yml'

In [38]:
loader_kwargs = { "undefined": StrictUndefined }

In [42]:
env = Environment(loader=FileSystemLoader(os.path.join('./test', 'fixtures')), **loader_kwargs)

In [43]:
template_context = { "vars": config["vars"] }

In [45]:
try:
    print(env.get_template(template).render(**template_context))
except TemplateNotFound:
    raise HokusaiError("Could not find template %s" % template)

UndefinedError: 'dict object' has no attribute 'artsyNetWildcardSSLCertz'